In [4]:
from tqdm.notebook import tqdm as tn
from tqdm import tqdm
import random
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
seed = 47
tqdm.pandas()
random.seed(seed)

In [5]:
cheques = pd.read_csv('data/cheques_public.csv', sep=';')
distances = np.array(pd.read_csv('data/darkstore.csv'))
darkstore_map = pd.read_csv('data/darkstore_map.csv', sep=';')
submit_map = pd.read_csv('baseline_submit.csv')
darkstore_map_genetic = pd.read_csv('data/sub_genetic_new_2.csv')
optuna_map = pd.read_csv('data/optuna_6_features.csv')
genetic_optuna_map = pd.read_csv('data/sub_genetic_optuna.csv')
sub_genetic_new_map = pd.read_csv('data/sub_genetic_new_3.csv')
optuna_clusters_map = pd.read_csv('data/optuna_clusters_features.csv')
new_map = pd.read_csv('data/sub_final.csv')

In [6]:
picking_time_speed = {1: 1, 2: 2, 3: 3}

In [7]:
def validator(cheques, distances):
    
    all_times = []
    for i, temp_cheque in cheques.groupby('CHEQUEID'):

        sum_time = 0
        current_location = 0
        #print('CHEQUE', cheque)
        #temp_cheque = cheques[cheques['CHEQUEID']==cheque]
        sum_time += (temp_cheque['KOLVO'] * temp_cheque['LEVEL'].map(picking_time_speed)).sum()
        
        est_locatsii = True
        set_locations = set(temp_cheque['SECTION'])
        while est_locatsii:
            dists = sorted([(x, distances[current_location, x]) for x in set_locations], key=lambda x: x[1], reverse=False)
            current_location = dists[0][0]
            travel_time = dists[0][1] * 2
            sum_time += travel_time

            set_locations.remove(current_location)
            if not len(set_locations):
                est_locatsii = False

        dist_to_final = distances[0, current_location]
        sum_time += dist_to_final * 2
        
        all_times.append(sum_time)
        
    return np.mean(all_times), all_times

# Validation

In [8]:
from sklearn.model_selection import KFold, GroupKFold

kfold = KFold(n_splits=5, random_state=seed, shuffle=False)
group_kfold = GroupKFold(n_splits=5)
res = []
for i, (tdx, vdx) in enumerate(group_kfold.split(cheques, cheques['KOLVO'], cheques['CHEQUEID'])):
    train_cheques = cheques.iloc[tdx]
    valid_cheques = cheques.iloc[vdx]
    
    darkstore_map_fold = new_map.copy()
    darkstore_map_dict = darkstore_map_fold.set_index(['SECTION', 'LEVEL']).to_dict('index')
    darkstore_map_dict_inverse = dict()
    for k, v in darkstore_map_dict.items():
        darkstore_map_dict[k] = v['LAGERID']
        darkstore_map_dict_inverse[v['LAGERID']] = k
    
    
    train_cheques['LOCATION'] = train_cheques['LAGERID'].apply(lambda x: darkstore_map_dict_inverse[x])
    train_cheques[['SECTION', 'LEVEL']] = pd.DataFrame(train_cheques['LOCATION'].tolist(), index=train_cheques.index)
    valid_cheques['LOCATION'] = valid_cheques['LAGERID'].apply(lambda x: darkstore_map_dict_inverse[x])
    valid_cheques[['SECTION', 'LEVEL']] = pd.DataFrame(valid_cheques['LOCATION'].tolist(), index=valid_cheques.index)
    #folds[f'{i+1}_fold_optuna_map'] = darkstore_map_fold['LAGERID']
    train_mean, _ = validator(train_cheques, distances)
    valid_mean, _ = validator(valid_cheques, distances)
    print(f'{i} Fold. train_mean: {train_mean}, valid_mean: {valid_mean}')
    res.append(valid_mean)
print('AVG', np.mean(res))

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_split.py:102: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

/opt/conda/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0 Fold. train_mean: 64.80721339630743, valid_mean: 65.03092783505154


/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_split.py:102: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



1 Fold. train_mean: 64.8529411764706, valid_mean: 64.8479381443299


/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_split.py:102: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



2 Fold. train_mean: 64.8492915414341, valid_mean: 64.86254295532646


/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_split.py:102: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



3 Fold. train_mean: 64.81640541120893, valid_mean: 64.99399141630902


/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_split.py:102: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



4 Fold. train_mean: 64.93386300193258, valid_mean: 64.5244635193133
AVG 64.85197277406604


In [ ]:
# бейслайн популярности товаров
# 0 Fold. train_mean: 67.7400171747531, valid_mean: 67.76632302405498
# 1 Fold. train_mean: 67.65671962215544, valid_mean: 68.26546391752578
# 2 Fold. train_mean: 67.69042507513954, valid_mean: 68.15979381443299
# 3 Fold. train_mean: 68.06549280652781, valid_mean: 68.34334763948497
# 4 Fold. train_mean: 68.0360747262186, valid_mean: 67.84377682403434
# AVG 68.07574104390662
# на всем треине 67.71453108897286, 394234.0

# new_genetic
# 0 Fold. train_mean: 67.33984542722199, valid_mean: 66.92096219931271
# 1 Fold. train_mean: 67.21124946328896, valid_mean: 67.43556701030928
# 2 Fold. train_mean: 67.21404036066981, valid_mean: 67.42439862542955
# 3 Fold. train_mean: 67.19261327034572, valid_mean: 67.50987124463519
# 4 Fold. train_mean: 67.3227399613485, valid_mean: 66.98969957081545
# AVG 67.25609973010043

# new_genetic_v2
# 0 Fold. train_mean: 66.83039931300988, valid_mean: 66.57903780068729
# 1 Fold. train_mean: 66.70502361528553, valid_mean: 67.0807560137457
# 2 Fold. train_mean: 66.75976814083297, valid_mean: 66.86168384879726
# 3 Fold. train_mean: 66.72063560231909, valid_mean: 67.01802575107295
# 4 Fold. train_mean: 66.88490444492162, valid_mean: 66.36137339055794
# AVG 66.78017536097224

# optuna
# 0 Fold. train_mean: 65.22069557750108, valid_mean: 65.51030927835052
# 1 Fold. train_mean: 65.28016316015457, valid_mean: 65.27233676975945
# 2 Fold. train_mean: 65.26792614856161, valid_mean: 65.3213058419244
# 3 Fold. train_mean: 65.27678763152244, valid_mean: 65.28583690987125
# 4 Fold. train_mean: 65.34743397036719, valid_mean: 65.00343347639485
# AVG 65.27864445526009
# на всем треине 65.278, 380052.0

# дообучение вектора с оптуны генетическими алгоритами
# 0 Fold. train_mean: 64.95148132245599, valid_mean: 65.01202749140893
# 1 Fold. train_mean: 64.9776728209532, valid_mean: 64.90721649484536
# 2 Fold. train_mean: 64.95706311721769, valid_mean: 64.98969072164948
# 3 Fold. train_mean: 64.9418080309212, valid_mean: 65.05064377682403
# 4 Fold. train_mean: 64.98990766587931, valid_mean: 64.85836909871244
# AVG 64.96358951668805
# на всем треине 64.9635, 378218.0

# Big subsamples validation

In [9]:
from sklearn.model_selection import KFold, GroupKFold

num_cheques = cheques['CHEQUEID'].nunique()
global_results = []
for temp_seed in tn(range(10)):
    subsample = random.sample(list(cheques['CHEQUEID'].unique()), int(num_cheques * 0.1))
    valid_cheques = cheques[cheques['CHEQUEID'].isin(subsample)]
    d = dict()
    for name, ds_map in [('optuna_map', optuna_map),
                  ('genetic_optuna_map', genetic_optuna_map),
                  ('sub_genetic_new_map', sub_genetic_new_map),
                        ('new_map', new_map)]:

        darkstore_map_dict = ds_map.set_index(['SECTION', 'LEVEL']).to_dict('index')
        darkstore_map_dict_inverse = dict()
        for k, v in darkstore_map_dict.items():
            darkstore_map_dict[k] = v['LAGERID']
            darkstore_map_dict_inverse[v['LAGERID']] = k

        valid_cheques['LOCATION'] = valid_cheques['LAGERID'].apply(lambda x: darkstore_map_dict_inverse[x])
        valid_cheques[['SECTION', 'LEVEL']] = pd.DataFrame(valid_cheques['LOCATION'].tolist(), index=valid_cheques.index)
        valid_mean, _ = validator(valid_cheques, distances)
        d[name] = valid_mean
    global_results.append(d)

  0%|          | 0/10 [00:00<?, ?it/s]

In [10]:
def highlight_last_max(data, colormax='antiquewhite', colormaxlast='lightgreen'):
    colormax_attr = f'background-color: {colormax}'
    colormaxlast_attr = f'background-color: {colormaxlast}'
    max_value = data.min()
    is_max = [colormax_attr if v == max_value else '' for v in data]
    is_max[len(data) - list(reversed(data)).index(max_value) -  1] = colormaxlast_attr
    return is_max



In [11]:
res = pd.DataFrame(global_results)
res.style.apply(highlight_last_max,axis=1)

,optuna_map,genetic_optuna_map,sub_genetic_new_map,new_map
0,64.520619,66.920962,64.180412,63.900344
1,65.128866,67.390034,64.781787,64.709622
2,65.446735,67.524055,65.216495,64.718213
3,65.432990,67.938144,65.101375,65.111684
4,65.077320,67.429553,64.910653,64.922680
5,65.314433,68.067010,64.814433,64.891753
6,64.656357,66.572165,64.599656,63.908935
7,64.142612,66.496564,63.841924,63.352234
8,65.352234,66.934708,64.917526,65.054983
9,65.852234,67.821306,65.778351,65.049828


# Full predict

In [12]:
darkstore_map_dict = new_map.set_index(['SECTION', 'LEVEL']).to_dict('index')
darkstore_map_dict_inverse = dict()
for k, v in darkstore_map_dict.items():
    darkstore_map_dict[k] = v['LAGERID']
    darkstore_map_dict_inverse[v['LAGERID']] = k

In [13]:
cheques['LOCATION'] = cheques['LAGERID'].apply(lambda x: darkstore_map_dict_inverse[x])
cheques[['SECTION', 'LEVEL']] = pd.DataFrame(cheques['LOCATION'].tolist(), index=cheques.index)

In [14]:
mean, all_times = validator(cheques, distances)
mean

64.85194091377534

In [15]:
sum(all_times)

377568.0

# Notes

In [ ]:
# !pip install apyori

In [ ]:
# from apriori_python import apriori

# freqItemSet, rules = apriori(cheques.groupby('CHEQUEID')['LAGERID'].apply(list).tolist(), minSup=0.03, minConf=0.03)
# print(freqItemSet)
# print(rules)  

## Word2Vec

In [ ]:
cheques['LAGERID'] = cheques.apply(lambda x: [x['LAGERID']] * int(x['KOLVO']), axis=1)
cheques = cheques.explode('LAGERID')
cheques['LAGERID'] = cheques['LAGERID'].astype(int)

In [ ]:
from gensim.models import Word2Vec

cheques2list = cheques.groupby('CHEQUEID')['LAGERID'].apply(list).tolist()
model = Word2Vec(sentences=cheques2list, vector_size=16, window=3, min_count=1, workers=10, epochs=25)

In [ ]:
vectors = [model.wv[i] for i in cheques['LAGERID'].unique()]

In [ ]:
model.wv.most_similar(10, topn=10)

In [ ]:
from sklearn.cluster import DBSCAN

# clustering = DBSCAN(eps=0.03, min_samples=2, metric='cosine').fit(vectors)
clustering = DBSCAN(eps=0.3, min_samples=2, metric='cosine').fit(vectors)
item_to_cluster = dict()
for i, lagerid in enumerate(cheques['LAGERID'].unique()):
    item_to_cluster[lagerid] = clustering.labels_[i]


In [ ]:
from collections import Counter
Counter(clustering.labels_).most_common()

In [ ]:
pd.to_pickle(item_to_cluster, 'lager_to_cluster.pkl')

## Stack

In [ ]:
darkstore_map_genetic = pd.read_csv('data/sub_genetic_new_2.csv')
optuna_map = pd.read_csv('data/optuna_6_features.csv')
genetic_optuna_map = pd.read_csv('data/sub_genetic_optuna.csv')

In [ ]:
folds = darkstore_map_genetic.copy().drop(columns=['LAGERID'])
folds['closest_popular'] = pd.Categorical(folds['SECTION'], distances[0].argsort())
folds['sub1'] = darkstore_map_genetic['LAGERID']
folds['sub2'] = optuna_map['LAGERID']
folds['sub3'] = genetic_optuna_map['LAGERID']

folds = folds.sort_values('closest_popular')

In [ ]:
from scipy import stats

ex_mode = set()
target = []
for t, x in folds.iterrows():
    mode = int(stats.mode([x['sub1'], x['sub2'], x['sub3']])[0][0])
    if mode in ex_mode:
        target.append(np.nan)
    else:
        target.append(mode)
        ex_mode.add(mode)
   

In [ ]:
folds['predict'] = target

In [ ]:
for_gridsearch = set(folds['sub1']) - set(folds.dropna()['predict'])

In [ ]:
from itertools import permutations

best_pair = (100, [])
for i, pair in tn(enumerate(permutations(for_gridsearch))):
    
    folds['LAGERID'] = folds['predict']
    folds.loc[folds[folds['LAGERID'].isna()==True].index, 'LAGERID'] = list(pair)
    darkstore_map_dict = folds.set_index(['SECTION', 'LEVEL']).to_dict('index')
    darkstore_map_dict_inverse = dict()
    for k, v in darkstore_map_dict.items():
        darkstore_map_dict[k] = v['LAGERID']
        darkstore_map_dict_inverse[v['LAGERID']] = k
    cheques['LOCATION'] = cheques['LAGERID'].apply(lambda x: darkstore_map_dict_inverse[x])
    cheques[['SECTION', 'LEVEL']] = pd.DataFrame(cheques['LOCATION'].tolist(), index=cheques.index)
    mean_time, _ = validator(cheques, distances)
    if mean_time<best_pair[0]:
        best_pair = (mean_time, pair)
    print(mean_time)